In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Significance filtering

After dropout filtering is done, we proceed to filtering by the **KS statistical test**.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from local_utils import kstest_function, pca_comp
import seaborn as sns

Our goal is to filter out the gens that have only few conditions that are non-zero as well as genes that have almost constant expression among the conditions.

The dataframe df_nonzero_means contains only the means of the replicates for each sample, while the df_nonzero_reps contains only the replicates of each sampple.

In [12]:
df_nonzero_means = pd.read_csv("df_nonzero_means.csv", sep="\t")
df_nonzero_reps = pd.read_csv("df_nonzero_reps.csv", sep="\t")

In [13]:
# put all sen first, all non-sen and basal last
dsen = df_nonzero_reps.filter(regex='FSen|SSen|MSen')
dnsen = df_nonzero_reps.filter(regex='F_|S_|M_|FNSen|SNSen|MNSen')
dtest = dsen.join(dnsen)

In [14]:
dtest["pval"] = dtest.apply(kstest_function, axis = 1)
bonferroni = 0.05/len(df_nonzero_means)
d_new = dtest.loc[dtest['pval'] <=  bonferroni]

The d_new dataframe contains the 142 genes selected by the KS test. However, this dataframe contains the 3 replicates, now we want to extract the given indices from the df_nonzero_means dataframe.

In [15]:
df_filteredKS = df_nonzero_means.loc[d_new.index, :]
df_filteredKS.to_csv("df_filteredKS.csv", sep = "\t", index = False)

As a control, if we randomly resample the conditions, we get no significant genes:

In [16]:
df_resampling = df_nonzero_reps.drop(columns = 'geneID', axis = 1, inplace= False)

df_random36 = df_resampling.sample(n=36,axis='columns')
df_random71 = df_resampling[df_resampling.columns.difference(df_random36.columns)] 

dtest_random = df_random36.join(df_random71)
dtest_random["pval"] = dtest_random.apply(kstest_function, axis = 1)

(dtest_random['pval'] <= bonferroni).value_counts() 

pval
False    28603
Name: count, dtype: int64